In [1]:
from invisible_cities. io. mcinfo_io import load_mchits
from invisible_cities. io. mcinfo_io import load_mcparticles
import tables as tb
from collections import defaultdict

In [2]:
in_path = "/Users/ansimez/Documents/InvCit/hdf5/InternalCalib/InternalCalib_Tl208.next.h5"
testfile = tb.open_file(in_path)

In [3]:
hits = load_mchits(in_path)

In [4]:
particles = load_mcparticles(in_path)

In [197]:
particles[2][4]

 MCParticle: name = e-
                    initial volume = CATHODE_GRID, final volume = BUFFER
                    kinetic energy = 0.39421600103378296 MeV
                    mother = 1, creator process = RadioactiveDecay
                    number of hits = 43
                    

In [8]:
len(hits.keys())

8305

In [13]:
particles[0]

{1:  MCParticle: name = Tl208
                     initial volume = CATHODE_GRID, final volume = CATHODE_GRID
                     kinetic energy = 0.0 MeV
                     mother = 0, creator process = none
                     number of hits = 0
                     , 2:  MCParticle: name = Pb208[3197.711]
                     initial volume = CATHODE_GRID, final volume = CATHODE_GRID
                     kinetic energy = 5.970476195216179e-06 MeV
                     mother = 1, creator process = RadioactiveDecay
                     number of hits = 0
                     , 3:  MCParticle: name = anti_nu_e
                     initial volume = CATHODE_GRID, final volume = WORLD
                     kinetic energy = 0.4570920467376709 MeV
                     mother = 1, creator process = RadioactiveDecay
                     number of hits = 0
                     , 4:  MCParticle: name = e-
                     initial volume = CATHODE_GRID, final volume = BUFFER
             

In [194]:
print(particles[0][4].initial_volume)

CATHODE_GRID


In [51]:
energy = 0.
for particle in particles[41].values():
    if particle.mother_indx == 2:
        energy += particle.E

In [52]:
energy

0.27735322521766648

In [65]:
t_0 = particles[12][14].final_vertex[3]

for particle in particles[12].values():
    if particle.mother_indx == 14:
        print(particle.initial_vertex[3]-t_0)

0.0
0.0
0.0
0.0
0.0


In [100]:
particles[80]

{1:  MCParticle: name = Tl208
                     initial volume = CATHODE_GRID, final volume = CATHODE_GRID
                     kinetic energy = 0.0 MeV
                     mother = 0, creator process = none
                     number of hits = 0
                     , 2:  MCParticle: name = Pb208[3197.711]
                     initial volume = CATHODE_GRID, final volume = CATHODE_GRID
                     kinetic energy = 7.885333616286516e-06 MeV
                     mother = 1, creator process = RadioactiveDecay
                     number of hits = 0
                     , 3:  MCParticle: name = anti_nu_e
                     initial volume = CATHODE_GRID, final volume = WORLD
                     kinetic energy = 0.6744921803474426 MeV
                     mother = 1, creator process = RadioactiveDecay
                     number of hits = 0
                     , 4:  MCParticle: name = e-
                     initial volume = CATHODE_GRID, final volume = BUFFER
             

In [150]:
n_beta       = 0
n_gamma_high = 0
n_pair       = 0

p_init_active = defaultdict(list)
p_end_active  = defaultdict(list)
p_full_active = defaultdict(list)

dt = []

g_511_init_active  = defaultdict(list)
g_583_init_active  = defaultdict(list)
g_763_init_active  = defaultdict(list)
g_277_init_active  = defaultdict(list)
g_860_init_active  = defaultdict(list)
g_2600_init_active = defaultdict(list)
g_all_init_active  = defaultdict(list)

for key, evt_particles in particles.items():
    n_gamma_high_evt = 0
    high_gamma_index = 0
    pp_particles = []
    for part_id, particle in evt_particles.items():

        if particle.initial_volume == 'ACTIVE':
            p_init_active[key].append(particle)
            
            if particle.name=='gamma' :
                partE = particle.E
                g_all_init_active[key].append(particle)
                if 0.275 < partE < 0.279:
                    g_277_init_active[key].append(particle)
                if 0.509 < partE < 0.513:
                    g_511_init_active[key].append(particle)
                if 0.581 < partE < 0.585:
                    g_583_init_active[key].append(particle)
                if 0.761 < partE < 0.765:
                    g_763_init_active[key].append(particle)
                if 0.858 < partE < 0.862:
                    g_860_init_active[key].append(particle)
                if 2.610 < partE < 2.618:
                    g_2600_init_active[key].append(particle)
                    
            if particle.final_volume   == 'ACTIVE':
                p_full_active[key].append(particle)
        if particle.final_volume   == 'ACTIVE':
            p_end_active[key].append(particle)

            
            
#        sel = (particle.process        == 'RadioactiveDecay') &\
#              (particle.initial_volume == 'CATHODE_GRID'    ) &\
#              (particle.final_volume   == 'ACTIVE'          )
#                if sel:
#                    n_active = 

#        if sel:            
#            if (particle.name == 'e-'   ) & (particle.E > 1.2):
#                n_beta += 1
#            if (particle.name == 'gamma'):# & (particle.E > 2.5):
#                n_gamma_high_evt += 1
#                n_gamma_high     += 1
        
        sel = (particle.name           == 'e+'    ) &\
              (particle.initial_volume == 'ACTIVE') &\
              (particle.final_volume   == 'ACTIVE') &\
              (particle.process        == 'conv'  )

        if sel:
           pp_particles.append(particle)
        
        sel = (particle.name           == 'e-'    ) &\
              (particle.initial_volume == 'ACTIVE') &\
              (particle.final_volume   == 'ACTIVE') &\
              (particle.process        == 'conv'  )

        if sel:
           pp_particles.append(particle)
    dt.append(t_max-t_min)

    for i, particle1 in enumerate(pp_particles):
        for j, particle2 in enumerate(pp_particles):
            if (particle1.name != particle2.name) & (particle1.mother_indx == particle2.mother_indx):
                n_pair += 1

print(f"Beta      interactions in active = {n_beta}")
print(f"Gamma 2.6 interactions in active = {n_gamma_high}")

print(f"PP        interactions in active = {n_pair/2}")

Beta      interactions in active = 0
Gamma 2.6 interactions in active = 0
PP        interactions in active = 21.0


In [203]:
s2s = []
s2s_z = []
for hit in hits[0]
    if 

if hit.Z 

SyntaxError: invalid syntax (<ipython-input-203-c07dcc29c0d6>, line 1)

In [202]:
hits[0][2]

<label = ACTIVE, pos = [-89.72525787353516, 74.66645812988281, 529.5132446289062], E = 0.0036452640779316425, time = 155957936128.0>

In [201]:
particles[1]

{1:  MCParticle: name = Tl208
                     initial volume = CATHODE_GRID, final volume = CATHODE_GRID
                     kinetic energy = 0.0 MeV
                     mother = 0, creator process = none
                     number of hits = 0
                     , 2:  MCParticle: name = Pb208[3197.711]
                     initial volume = CATHODE_GRID, final volume = CATHODE_GRID
                     kinetic energy = 1.108401920646429e-05 MeV
                     mother = 1, creator process = RadioactiveDecay
                     number of hits = 0
                     , 3:  MCParticle: name = anti_nu_e
                     initial volume = CATHODE_GRID, final volume = WORLD
                     kinetic energy = 1.3143500089645386 MeV
                     mother = 1, creator process = RadioactiveDecay
                     number of hits = 0
                     , 4:  MCParticle: name = e-
                     initial volume = CATHODE_GRID, final volume = ACTIVE
             

In [175]:
n_beta       = 0
n_gamma_high = 0
n_pair       = 0

p_init_active = defaultdict(list)
p_end_active  = defaultdict(list)
p_full_active = defaultdict(list)

dt = []

g_511_init_active  = defaultdict(list)
g_583_init_active  = defaultdict(list)
g_763_init_active  = defaultdict(list)
g_277_init_active  = defaultdict(list)
g_860_init_active  = defaultdict(list)
g_2600_init_active = defaultdict(list)
g_all_init_active = defaultdict(list)

b_all        = defaultdict(dict)
b_end_active = defaultdict(dict)
b_end_buffer = defaultdict(dict)

g_all        = defaultdict(dict)
g_end_active = defaultdict(dict)
g_end_buffer = defaultdict(dict)

n_positron = 0

for key, evt_particles in particles.items():
    pp_particles = []
    for part_id, particle in evt_particles.items():
        sel = (particle.process        == 'RadioactiveDecay') &\
              (particle.initial_volume == 'CATHODE_GRID'    )
                
        if sel:
            pName        = particle.name
            pFinalVolume = particle.final_volume
            if pName == 'e-':
                b_all[key][part_id] = particle
                if pFinalVolume == 'ACTIVE':
                    b_end_active[key][part_id] = particle
                if pFinalVolume == 'BUFFER':
                    b_end_buffer[key][part_id] = particle
            if pName == 'gamma':
                g_all[key][part_id] = particle
                if pFinalVolume == 'ACTIVE':
                    g_end_active[key][part_id] = particle
                if pFinalVolume == 'BUFFER':
                    g_end_buffer[key][part_id] = particle

        sel = (particle.name           == 'e+'    ) &\
              ((particle.initial_volume == 'ACTIVE') or\
              (particle.final_volume   == 'ACTIVE'))
        if sel:
            n_positron += 1
                    
        sel = (particle.name           == 'e+'    ) &\
              (particle.initial_volume == 'ACTIVE') &\
              (particle.final_volume   == 'ACTIVE') &\
              (particle.process        == 'conv'  )

        if sel:
            pp_particles.append(particle)
        
        sel = (particle.name           == 'e-'    ) &\
              (particle.initial_volume == 'ACTIVE') &\
              (particle.final_volume   == 'ACTIVE') &\
              (particle.process        == 'conv'  )

        if sel:
            pp_particles.append(particle)
    
    for i, particle1 in enumerate(pp_particles):
        for j, particle2 in enumerate(pp_particles):
            if (particle1.name != particle2.name) & (particle1.mother_indx == particle2.mother_indx):
                n_pair += 1

print(f"PP        interactions in active = {n_pair/2}")

e_g_all        = defaultdict(dict)
e_g_end_active = defaultdict(dict)
e_g_end_buffer = defaultdict(dict)

for key, evt_particles in particles.items():        
    for part_id, particle in evt_particles.items():    
        if particle.mother_indx in g_all[key]:
            e_g_all[key][part_id] = particle
        if particle.mother_indx in g_end_active[key]:
            e_g_end_active[key][part_id] = particle            
        if particle.mother_indx in g_end_buffer[key]:
            e_g_end_buffer[key][part_id] = particle            
            
#        sel = (particle.process        == 'RadioactiveDecay') &\
#              (particle.initial_volume == 'CATHODE_GRID'    )

PP        interactions in active = 21.0


In [180]:
g_end_active[2]

{6:  MCParticle: name = gamma
                     initial volume = CATHODE_GRID, final volume = ACTIVE
                     kinetic energy = 0.5107393264770508 MeV
                     mother = 2, creator process = RadioactiveDecay
                     number of hits = 2
                     }

In [186]:
particles[2]

{1:  MCParticle: name = Tl208
                     initial volume = CATHODE_GRID, final volume = CATHODE_GRID
                     kinetic energy = 0.0 MeV
                     mother = 0, creator process = none
                     number of hits = 0
                     , 2:  MCParticle: name = Pb208[3708.451]
                     initial volume = CATHODE_GRID, final volume = CATHODE_GRID
                     kinetic energy = 5.089212208986282e-06 MeV
                     mother = 1, creator process = RadioactiveDecay
                     number of hits = 0
                     , 3:  MCParticle: name = anti_nu_e
                     initial volume = CATHODE_GRID, final volume = WORLD
                     kinetic energy = 0.8962079286575317 MeV
                     mother = 1, creator process = RadioactiveDecay
                     number of hits = 0
                     , 4:  MCParticle: name = e-
                     initial volume = CATHODE_GRID, final volume = BUFFER
             

In [191]:
hits[0]

[<label = ACTIVE, pos = [-88.988525390625, 75.36749267578125, 531.2091674804688], E = 0.004831037949770689, time = 155957936128.0>,
 <label = ACTIVE, pos = [-89.36878204345703, 74.96273040771484, 530.3846435546875], E = 0.003465907881036401, time = 155957936128.0>,
 <label = ACTIVE, pos = [-89.72525787353516, 74.66645812988281, 529.5132446289062], E = 0.0036452640779316425, time = 155957936128.0>,
 <label = ACTIVE, pos = [-90.26802062988281, 74.36997985839844, 528.7365112304688], E = 0.005101602058857679, time = 155957936128.0>,
 <label = ACTIVE, pos = [-90.91002655029297, 74.32123565673828, 527.9773559570312], E = 0.00827684998512268, time = 155957936128.0>,
 <label = ACTIVE, pos = [-91.6059341430664, 74.42821502685547, 527.27685546875], E = 0.010120509192347527, time = 155957936128.0>,
 <label = ACTIVE, pos = [-92.3224105834961, 74.69345092773438, 526.6395263671875], E = 0.0014230763772502542, time = 155957936128.0>,
 <label = ACTIVE, pos = [-93.22364044189453, 74.80754089355469, 526

In [189]:
particles[2][29].hits

[]

In [185]:
e_g_end_active[2]

{20:  MCParticle: name = e-
                     initial volume = ACTIVE, final volume = ACTIVE
                     kinetic energy = 0.09928818792104721 MeV
                     mother = 6, creator process = compt
                     number of hits = 8
                     , 21:  MCParticle: name = e-
                     initial volume = DRIFT_TUBE, final volume = DRIFT_TUBE
                     kinetic energy = 0.23720794916152954 MeV
                     mother = 6, creator process = compt
                     number of hits = 0
                     , 22:  MCParticle: name = e-
                     initial volume = DRIFT_TUBE, final volume = DRIFT_TUBE
                     kinetic energy = 0.06261883676052094 MeV
                     mother = 6, creator process = compt
                     number of hits = 0
                     , 23:  MCParticle: name = e-
                     initial volume = ACTIVE, final volume = ACTIVE
                     kinetic energy = 0.07629309594631195

In [183]:
energy = 0
for p in e_g_end_active[2].values():
    energy += p.E

In [184]:
energy

0.50518937036395073

In [174]:
g_all[2]

{6:  MCParticle: name = gamma
                     initial volume = CATHODE_GRID, final volume = ACTIVE
                     kinetic energy = 0.5107393264770508 MeV
                     mother = 2, creator process = RadioactiveDecay
                     number of hits = 2
                     , 8:  MCParticle: name = gamma
                     initial volume = CATHODE_GRID, final volume = ICS
                     kinetic energy = 0.5831881165504456 MeV
                     mother = 5, creator process = RadioactiveDecay
                     number of hits = 0
                     , 10:  MCParticle: name = gamma
                     initial volume = CATHODE_GRID, final volume = ICS
                     kinetic energy = 2.614504337310791 MeV
                     mother = 7, creator process = RadioactiveDecay
                     number of hits = 0
                     }

In [162]:
n_positron

38

In [153]:
len(g_583_init_active)

1

In [156]:
len(b_end_active)

4761

In [157]:
len(g_end_active)

1647

In [138]:
len(p_full_active)

4924

In [112]:
particles[0]

{1:  MCParticle: name = Tl208
                     initial volume = CATHODE_GRID, final volume = CATHODE_GRID
                     kinetic energy = 0.0 MeV
                     mother = 0, creator process = none
                     number of hits = 0
                     , 2:  MCParticle: name = Pb208[3197.711]
                     initial volume = CATHODE_GRID, final volume = CATHODE_GRID
                     kinetic energy = 5.970476195216179e-06 MeV
                     mother = 1, creator process = RadioactiveDecay
                     number of hits = 0
                     , 3:  MCParticle: name = anti_nu_e
                     initial volume = CATHODE_GRID, final volume = WORLD
                     kinetic energy = 0.4570920467376709 MeV
                     mother = 1, creator process = RadioactiveDecay
                     number of hits = 0
                     , 4:  MCParticle: name = e-
                     initial volume = CATHODE_GRID, final volume = BUFFER
             

In [129]:
p_init_active[12]

[ MCParticle: name = e-
                     initial volume = ACTIVE, final volume = ACTIVE
                     kinetic energy = 0.21114207804203033 MeV
                     mother = 4, creator process = eIoni
                     number of hits = 37
                     ]

In [134]:
particles[12][5].final_vertex

array([ -4.46209450e+01,  -1.21228226e+02,   5.24362671e+02,
         7.26835036e+10], dtype=float32)

In [128]:
len(p_init_active)

974

In [108]:
len(p_end_active)

6645

In [119]:
len(p_full_active)

778

In [116]:
p_full_active_E_cut = {k:p for k,p in p_full_active.items() if p.E > 0.1}

AttributeError: 'list' object has no attribute 'E'

In [98]:
len(p_full_active_E_cut)

2341

In [99]:
p_full_active_E_cut

[ MCParticle: name = e-
                     initial volume = ACTIVE, final volume = ACTIVE
                     kinetic energy = 0.12560386955738068 MeV
                     mother = 4, creator process = eIoni
                     number of hits = 43
                     ,  MCParticle: name = e-
                     initial volume = ACTIVE, final volume = ACTIVE
                     kinetic energy = 0.21114207804203033 MeV
                     mother = 4, creator process = eIoni
                     number of hits = 37
                     ,  MCParticle: name = e-
                     initial volume = ACTIVE, final volume = ACTIVE
                     kinetic energy = 0.10989142954349518 MeV
                     mother = 4, creator process = eIoni
                     number of hits = 12
                     ,  MCParticle: name = e-
                     initial volume = ACTIVE, final volume = ACTIVE
                     kinetic energy = 0.14898937940597534 MeV
                     mot

In [92]:
len(p_full_active)

16138

In [57]:
print(n_gamma_high)

305
